Import [GHCNd](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) data from its [NOAA Archieve](https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/archive/)   
<!-- Station data - US from 1950 to 2023 -->

* [Country code](https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-countries.txt)
* Station metadata

In [1]:
from pySmartDL import SmartDL
from os.path import basename
import tarfile
import os
from pymongo import MongoClient
import json
import requests
import pandas as pd
from io import StringIO
import numpy as np
from datetime import datetime
# import multiprocessing
# from io import BytesIO




## Get meta-data
def check_data_exists(name_db: str, name_collection: str):
    # Connect to MongoDB
    client = MongoClient('localhost', 27017)
    # db contains collections
    db = client[name_db]
    # mcd will save in each year
    collection = db[name_collection]

    # Check if collection contains data with "year" variable
    if collection.count_documents( {} ) > 0:
        print(f"DB:{name_db}, The {name_collection} collection contains data.")
        # no need update, retrun empty collection
        collection = {}
        return collection
    else:
        print(f"DB:{name_db}, The {name_collection} collection does not contain data.")
        return collection


def get_ghcnd_meta(url_meta, fwf_colspec, colnames):
    res_station = requests.get(url_meta)
    df_station = pd.read_fwf(
            StringIO(res_station.text),
            colspecs=fwf_colspec,
            names=colnames,
            )
    return df_station




def download_and_import_to_mongo(url, db_name, collection_name):
    # Download the file using pySmartDL
    dest = basename(url)
    obj = SmartDL(url, dest)
    obj.start()
    
    # Extract the .tar.gz file
    extracted_folder = dest.replace('.tar.gz', '')
    with tarfile.open(dest, 'r:gz') as tar:
        tar.extractall(extracted_folder)
    
    # Connect to MongoDB
    client = MongoClient('localhost', 27017)
    db = client[db_name]
    collection = db[collection_name]
    
    # Import the .json file(s) to MongoDB
    # (Assuming there's a .json file inside the extracted folder)
    for root, dirs, files in os.walk(extracted_folder):
        for filename in files:
            if filename.endswith('.json'):
                filepath = os.path.join(root, filename)
                
                # Open the .json file and read the data
                with open(filepath, 'r') as f:
                    data = json.load(f)
                    
                    # If data is a list of dictionaries, we can insert them all at once
                    if isinstance(data, list):
                        collection.insert_many(data)
                    else:
                        collection.insert_one(data)
    
    # Clean up (Optional: remove the downloaded and extracted files)
    os.remove(dest)
    os.rmdir(extracted_folder)


## Get station metadata if not have

In [2]:
# station meta
collection = check_data_exists('ghcnd', 'meta_station')
if not collection == {} :
    print('Get the station meta data')
    url_meta_station = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'
    fwf_colspec_station = [(0, 12), (12, 21), (21, 31), (31, 38), (38, 69)]
    col_names = ["station", "lat", "lon", "elev", "name"]
    df = get_ghcnd_meta(url_meta_station, fwf_colspec_station, col_names)
    # 
    records = df.to_dict('records')
    collection.insert_many(records)



# station meta - inventory
collection = check_data_exists('ghcnd', 'meta_station_inventory')
if not collection == {} :
    print('Get the station meta - inventory data')
    url_meta_station = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt'
    fwf_colspec_station = [(0, 12), (35, 41), (41, 46)]
    col_names = ["station", "start_year", "end_year"]
    df = get_ghcnd_meta(url_meta_station, fwf_colspec_station, col_names)
    # 
    records = df.to_dict('records')
    collection.insert_many(records)


DB:ghcnd, The meta_station collection contains data.
DB:ghcnd, The meta_station_inventory collection contains data.


In [5]:
# # ftp readme file
# from ftplib import FTP

# # FTP server details
# ftp_server = 'ftp.ncdc.noaa.gov'
# file_path = '/pub/data/ghcn/daily/readme.txt'
# local_file = 'downloaded_file.txt'

# # ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

# # Connect to the FTP server
# ftp = FTP(ftp_server)
# ftp.login()  # login with default user anonymous and empty password

# # Set the file mode to binary
# ftp.sendcmd("TYPE i")

# # Retrieve the file
# with open(local_file, 'wb') as local_file_handle:
#     ftp.retrbinary(f'RETR {file_path}', local_file_handle.write)

# # Close the connection
# ftp.quit()

# print(f'The file has been downloaded as: {local_file}')


### Select stations between 1982 to 2003 US

In [38]:
client = MongoClient('localhost', 27017)
db = client['ghcnd']



df_stations_8203 = pd.DataFrame(
        db['meta_station_inventory'].find(
                    {   'start_year' : {'$lte'   : 1982},
                        'end_year'   : {'$gte'   : 1982},
                        'station'    : {'$regex' : '^US'}
                            })
    ).drop(columns='_id')

# print(df_stations_8203.head())
# station list also be used to get the file.
stationlist = df_stations_8203['station'].unique().tolist()
# 

# 
df_stations_8203_meta = pd.DataFrame(
        db['meta_station'].find(
                    {'station' : {'$in':stationlist}}
        )
    ).drop(columns='_id')
# print(df_stations_8203_meta.head())

df_stations_8203_meta


,station,lat,lon,elev,name
0,USC00010008,31.5703,-85.2483,139.0,AL ABBEVILLE
1,USC00010063,34.2108,-87.1783,239.6,AL ADDISON
2,USC00010140,32.2322,-87.4103,53.3,AL ALBERTA
3,USC00010160,32.9350,-85.9556,201.2,AL ALEXANDER CITY
4,USC00010178,33.1272,-88.1550,59.4,AL ALICEVILLE
...,...,...,...,...,...
9650,USW00094911,42.8786,-97.3636,357.2,SD YANKTON 2 E
9651,USW00094918,41.3536,-96.0233,390.1,NE OMAHA #1
9652,USW00094931,47.3803,-92.8325,408.1,MN HIBBING CHISHOLM HIBBING AP
9653,USW00094957,40.0792,-95.5892,298.7,NE FALLS CITY BRENNER FLD


## get the observation data.
* Selecting stations by
    * Year 1982 to 2003, US

In [ ]:
'''ID = 11 character station identification code
YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
ELEMENT = 4 character indicator of element type 
DATA VALUE = 5 character data value for ELEMENT 
M-FLAG = 1 character Measurement Flag 
Q-FLAG = 1 character Quality Flag 
S-FLAG = 1 character Source Flag 
OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am); if no ob time information '''

In [36]:
def read_i_station(fname):
    col_dtypes = {
    'station': str,
    'elem': str,
    'value': np.float64,
    'flag_m': str,
    'flag_q': str,
    'flag_s': str,
    'time': np.float64
    }
    col_names = ['station', 'date', 'elem', 'value', 'flag_m', 'flag_q', 'flag_s', 'time']
    # 
    df = pd.read_csv(fname,
        compression='gzip',
        header=None,
        names=col_names,
        dtype=col_dtypes,
        parse_dates=['date'],
    )
    # 
    return df

# select measured element only in 
elem_list = [   'PRCP', # Precipitation (tenths of mm)
                'TMAX', # Maximum temperature (tenths of degrees C)
                'TMIN', # Minimum temperature (tenths of degrees C)
                'AWND', # Average daily wind speed (tenths of meters per second)

                'ACMC', # Average cloudiness midnight to midnight from 30-second c eilometer data (percent)
                'ACSC', # Average cloudiness sunrise to sunset from 30-second c eilometer data (percent)
                'RHAV', # Average relative humidity for the day (percent)
                'RHMN', # Minimum relative humidity for the day (percent)
                'RHMX'] # Maximum relative humidity for the day (percent)



# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['ghcnd']

for year in range(1972,2013) : # getting +- 10years of climate data.
    # check collection is empty - use temp
    collection = db['station_data_{elem}'.format(elem='TMAX')]
    if collection.count_documents({'date': {
            '$gte': datetime(year, 1, 1),
            '$lte': datetime(year, 12, 31)
        }}) == 0 :

        # 
        url = f'https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/{year}.csv.gz'
        obj = SmartDL(url, progress_bar=False)
        # if file already exists, pass
        if not os.path.exists(obj.get_dest()) :
            obj.start()

        # Read the downloaded file into memory
        df = read_i_station(obj.get_dest())
        # subsetting stations in the station list.
        df = df.loc[df['station'].isin(stationlist)]


        for elem in elem_list :
            df_elem = df.loc[df['elem'] == elem].reset_index(drop=True)
            if df_elem.index.max() > 0 :
                # normalize value and change 'value' to element value.
                # e.g. TMAX divde by 10. 
                if elem in ['PRCP','TMAX','TMIN','AWND'] :
                    df_elem['value'] = df_elem['value'] / 10 
                else :
                    df_elem['value'] = df_elem['value'] / 100
                
                # rename
                df_elem = df_elem.rename(columns={'value':elem}).drop(columns='elem')
                

                # DB collection
                collection = db[f'station_data_{elem}']
                # check existance and resume.
                # 
                records = df_elem.to_dict('records')
                collection.insert_many(records)

        # clear memory
        df = {}
        df_elem = {}
        # delete temp file
        os.remove(obj.get_dest())



    